# Introduction

This notebook illustrates how to extra Open States Bills meta data, and scrape content for one of the states using the [GraphQL](https://docs.openstates.org/en/latest/api/v2/index.html) API v2 interface.

It does the following:

- Extracts durisdictions and sessions, so you know their names
- For one of these (Rhode island), it extract all bills meta data for a session
- Using the link for the bill in this data, it downloads the PDF and extracts text
- Saves all data to a Pandas dataframe for analysis

Note: In this notebook, we developed a simple scraper for getting Bill text for simplicity, but Open states [provide scrapers also](https://docs.openstates.org/en/latest/contributing/getting-started.html?highlight=scraping#running-our-first-scraper)

## Before you start

You will need to be provided an API key, and use it to populate the 'headers' variable below. To get this key, [register](https://openstates.org/api/register/).

# Setup

In [1]:
import pandas as pd
import os
import sys
import json
from newspaper import fulltext
from bs4 import BeautifulSoup
import re
from newspaper import Article
import PyPDF2
import wget
import requests
import pickle
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import traceback
import subprocess
import logging

# Openstates API key (they sent me this on request)
headers = {"PUT YOUR OPENSTATE API KEY HERE"}

# Analysys
## Finding jurisdictions and sessions

We are after bills which have been labelled with [Subjects](https://docs.openstates.org/en/latest/policies/categorization.html?highlight=categories#subjects) such as 'Agriculture and Food'. In the API I couldn't find any bills with this set, in either V1 or V2 API. So we'll scan a sample across all states, sessions to see if it's set anywhere

In [7]:
# Get durisdictions and sessions
params = {'query': '''
    {
      jurisdictions {
        edges {
          node {
            id
            name
            legislativeSessions {
              edges {
                node {
                  identifier
                  name
                  classification
                  startDate
                  endDate
                }
              }
            }
          }
        }
      }
    }
'''}

response = requests.post("https://openstates.org/graphql", params=params, headers=headers)
data = json.loads(response.content)
data = data["data"]["jurisdictions"]["edges"]
#display(data)
sessions = []
for d in data:
    d = d["node"]
    for s in d["legislativeSessions"]["edges"]:
        s = s["node"]
        s["legislature"] = d["name"]
        sessions.append(s)
    
sessions = pd.DataFrame(sessions)

# Filter for 2017-2018 (Note, for some states where startDate isn't populated, this excludes 2017 data)
sessions = sessions[sessions['startDate'].str.match('2017')]

sessions = sessions.sort_values(by='legislature')

display(sessions)

,classification,endDate,identifier,legislature,name,startDate
140,primary,2017-05-31,2017rs,Alabama,2017 Regular Session,2017-02-07
466,,2017-04-16,30,Alaska,30th Legislature (2017-2018),2017-01-17
29,primary,2017-05-03,53rd-1st-regular,Arizona,53rd Legislature - 1st Regular Session,2017-01-09
7,primary,2017-04-22,2017,Arkansas,2017 Regular Session,2017-01-12
9,special,2017-07-01,2017S1,Arkansas,2017 First Extraordinary Session,2017-05-01
156,special,2017-10-06,2017B,Colorado,2017 First Extraordinary Session,2017-10-02
155,primary,2017-05-10,2017A,Colorado,2017 Regular Session,2017-01-11
205,,2017-06-07,2017,Connecticut,2017 Regular Session,2017-01-04
194,,2017-06-30,149,Delaware,149th General Assembly (2017-2018),2017-01-10
295,,2017-12-31,22,District of Columbia,22nd Council Period (2017-2018),2017-01-01


## Finding sessions which have the 'subject' field 

Bills can have a 'Subject' set, which includes a list of tags such as 'Taxation' (see description in [API v1 documentation](https://docs.openstates.org/en/latest/policies/categorization.html?highlight=subject#subjects)). However, not all states have this set. The following code is a simple example of looping through jurisdictions and sessions and looking at a few bills in each to see if the subject field is set.

In [8]:
params_template = {'query': '''
            {
              search_1: bills(first: 4, session: "SESSION_NAME", jurisdiction: "JURISDICTION_NAME",classification:"bill") {
                edges {
                  node {
                    id
                    identifier
                    subject
                    title
                  }
                }
                pageInfo {
                  hasNextPage
                  hasPreviousPage
                  endCursor
                  startCursor
                }
                totalCount
              }
        }
'''}

counts = []
has_subjects = []
for index, row in sessions.iterrows():
    params = params_template.copy()
    state   = row["legislature"]
    session = row["identifier"]
    print("\n\nState: " + state + "; Session identifier: " + session)
    params["query"] = params["query"].replace("SESSION_NAME",session).replace("JURISDICTION_NAME", state).replace("\n","")
    response = requests.post("https://openstates.org/graphql", params=params, headers=headers)
    bills = json.loads(response.content)
    # Stash page Info, for pagination
    count = bills["data"]["search_1"]["totalCount"]
    counts.append(count)
    if count > 0:
        print(str(count) + " bills found")
        bills = bills["data"]["search_1"]["edges"]
        if len(bills[0]["node"]["subject"]) > 0:
            has_subjects.append("Yes")
            display(bills[0]["node"]["subject"])
        else:
            has_subjects.append("No")
            print("No subjects found")
    else:
        print("No bills found")
        has_subjects.append("No")
        
sessions["billsCount"]    = counts
sessions["has_subjects"]  = has_subjects

display(sessions)

print("Done")



State: Alabama; Session identifier: 2017rs
1030 bills found


['Constitutional Amendments']



State: Alaska; Session identifier: 30
628 bills found


['BONDS & BONDING',
 'ECONOMIC DEVELOPMENT',
 'ENERGY',
 'FUNDS',
 'OIL & GAS',
 'PUBLIC CORPORATIONS',
 'PUBLIC FINANCE']



State: Arizona; Session identifier: 53rd-1st-regular
1082 bills found


...



State: Wisconsin; Session identifier: January 2017 Special Session
22 bills found


['Mental health',
 'Special session, 2017 -- January',
 ', and its subheadings',
 'Public Instruction, Department of',
 'School -- Employee, see also ',
 'Governor -- Legislation by request of']



State: Wisconsin; Session identifier: 2017 Regular Session
1599 bills found
No subjects found


State: Wyoming; Session identifier: 2017
465 bills found
No subjects found


,classification,endDate,identifier,legislature,name,startDate,billsCount,has_subjects
140,primary,2017-05-31,2017rs,Alabama,2017 Regular Session,2017-02-07,1030,Yes
466,,2017-04-16,30,Alaska,30th Legislature (2017-2018),2017-01-17,628,Yes
29,primary,2017-05-03,53rd-1st-regular,Arizona,53rd Legislature - 1st Regular Session,2017-01-09,1082,Yes
7,primary,2017-04-22,2017,Arkansas,2017 Regular Session,2017-01-12,0,No
9,special,2017-07-01,2017S1,Arkansas,2017 First Extraordinary Session,2017-05-01,10,No
156,special,2017-10-06,2017B,Colorado,2017 First Extraordinary Session,2017-10-02,4,No
155,primary,2017-05-10,2017A,Colorado,2017 Regular Session,2017-01-11,793,No
205,,2017-06-07,2017,Connecticut,2017 Regular Session,2017-01-04,3387,No
194,,2017-06-30,149,Delaware,149th General Assembly (2017-2018),2017-01-10,746,No
295,,2017-12-31,22,District of Columbia,22nd Council Period (2017-2018),2017-01-01,3277,No


Done


## Extract Bills meta data and scrape text for Rhode Island

The following section uses the GraphQL API to extract all bills for a specific session for Rhode Island. It also scrapes bill text, but as mentioned above Open states [provide scrapers also](https://docs.openstates.org/en/latest/contributing/getting-started.html?highlight=scraping#running-our-first-scraper).

In [311]:
#
# A little parser of PDFs, tuned using Rhode Island bills, not tested on other states
#

def extractPDF(url):  
    filename = wget.download(url)
    pdfReader = PyPDF2.PdfFileReader(filename)
    count = pdfReader.numPages
    text = ""
    for i in range(count):
        page = pdfReader.getPage(i)
        text = text + page.extractText()
    # Remove lines which just have a number on them
    text = re.sub(r'^[0-9]*\n','', text, flags=re.M)
    text = re.sub(r'^\n','', text, flags=re.M)
    # Hyphenated words
    text = re.sub(r'\n^-','-', text, flags=re.M)
    text = re.sub(r'-( |)\n','-', text, flags=re.M)
    text = re.sub(r'\n','DOUBLE', text, flags=re.M)
    # Line breaks introduced by PDF parser
    text = re.sub(r'DOUBLE DOUBLE DOUBLE DOUBLE','\n\n', text, flags=re.M)
    text = re.sub(r'DOUBLE DOUBLE DOUBLE','\n\n', text, flags=re.M)
    text = re.sub(r'DOUBLE','', text, flags=re.M)
    os.remove(filename)
    return text

jurisdiction = "Rhode Island"
session      = "2017"

params_template = {'query': '''
            {
              search_1: bills(first: 4, session: "SESSION", jurisdiction: "JURISDICTION", classification:"bill" AFTER_CLAUSE) {
                edges {
                  node {
                    id
                    identifier
                    subject
                    title
                    abstracts {
                      abstract
                      note
                      date
                    }
                    openstatesUrl
                    sponsorships {
                      name
                      entityType
                      primary
                      classification
                    }
                    classification
                    subject
                    otherIdentifiers {
                      identifier
                      scheme
                      note
                    }
                    updatedAt
                    createdAt
                    legislativeSession {
                      identifier
                      jurisdiction {
                        name
                      }
                    }
                    actions {
                      date
                      description
                      classification
                    }
                    documents {
                      date
                      note
                      links {
                        url
                      }
                    }
                    versions {
                      date
                      note
                      links {
                        url
                      }
                    }
                    sources {
                      url
                      note

                    }
                  }
                }
                pageInfo {
                  hasNextPage
                  hasPreviousPage
                  endCursor
                  startCursor
                }
                totalCount
              }
        }
'''}

params = params_template.copy()
params["query"] = params["query"].replace("JURISDICTION",jurisdiction)
params["query"] = params["query"].replace("SESSION",session)
params_template = params.copy()

# Paginate API calls
hasNextPage = True
endCursor   = ""
bill_array  = []

count = 0

print("Starting API calls and PDF scraping for " + jurisdiction + " " + session + " ...")

while hasNextPage == True:
    
    params = params_template.copy()
    
    if endCursor != "":
        params["query"] = params["query"].replace("AFTER_CLAUSE",',after:"'+ endCursor + '"')
    else:
        params["query"] = params["query"].replace("AFTER_CLAUSE",'')
    
    response = requests.post("https://openstates.org/graphql", params=params, headers=headers)
    bills = json.loads(response.content)
    bills = bills["data"]["search_1"]

    # Stash page Info, for pagination
    pageInfo    = bills["pageInfo"]
    hasNextPage = pageInfo["hasNextPage"]
    if hasNextPage == True:
        endCursor = pageInfo["endCursor"]
    
    # Loop through results
    for r in bills["edges"]:
        b = r["node"]
        url = b["versions"][0]["links"][0]["url"] 
        #display(url)
        b["billText"] = extractPDF(url)
        #print(b["billText"])
        bill_array.append(b)
        count = count + 1
        
    if count % 100 == 0:
        print("Processed " + str(count) + " bills")

df = pd.DataFrame(bill_array)
display(df)

Starting API calls and PDF scraping for Rhode Island 2017 ...
Processed 100 bills
Processed 200 bills
Processed 300 bills
Processed 400 bills
Processed 500 bills
Processed 600 bills
Processed 700 bills
Processed 800 bills
Processed 900 bills
Processed 1000 bills
Processed 1100 bills
Processed 1200 bills
Processed 1300 bills
Processed 1400 bills
Processed 1500 bills
Processed 1600 bills
Processed 1700 bills
Processed 1800 bills
Processed 1900 bills


,abstracts,actions,billText,classification,createdAt,documents,id,identifier,legislativeSession,openstatesUrl,otherIdentifiers,sources,sponsorships,subject,title,updatedAt,versions
0,[],"[{'date': '2017-02-01', 'description': '02/01/...",\n\n -- S 0122 SUBSTITUTE A ======== LC000032...,[bill],2017-07-18 05:33:25.715117+00:00,[],ocd-bill/93da2fe4-89fe-4833-b688-21e0de1f5bf7,SB 122,"{'identifier': '2017', 'jurisdiction': {'name'...",https://openstates.org/ri/bills/2017/SB122,[],"[{'url': 'http://status.rilin.state.ri.us/', '...","[{'name': 'Crowley', 'entityType': 'person', '...","[Children and Families, Human Services, Public...",AN ACT RELATING TO HUMAN SERVICES -- CHILD CAR...,2017-12-30 05:39:44.167910+00:00,"[{'date': '', 'note': '122 SUB A', 'links': [..."
1,[],"[{'date': '2017-01-18', 'description': '01/18/...",\n\n -- H 5137 ======== LC000786 ======== STA...,[bill],2017-07-18 05:33:34.223355+00:00,[],ocd-bill/b7fbf3a1-19c5-4a98-a814-7d74afe4bf35,HB 5137,"{'identifier': '2017', 'jurisdiction': {'name'...",https://openstates.org/ri/bills/2017/HB5137,[],"[{'url': 'http://status.rilin.state.ri.us/', '...","[{'name': 'Bennett', 'entityType': 'person', '...","[Plates (M.V.), Motor Vehicles]",AN ACT RELATING TO MOTOR AND OTHER VEHICLES-SP...,2017-12-31 05:39:17.364624+00:00,"[{'date': '', 'note': '5137', 'links': [{'url'..."
2,[],"[{'date': '2017-01-11', 'description': '01/11/...",\n\n -- H 5067 ======== LC000199 ======== STA...,[bill],2017-07-18 05:33:37.271885+00:00,[],ocd-bill/797cbed6-8384-43d5-b071-bbe34897625d,HB 5067,"{'identifier': '2017', 'jurisdiction': {'name'...",https://openstates.org/ri/bills/2017/HB5067,[],"[{'url': 'http://status.rilin.state.ri.us/', '...","[{'name': 'Amore', 'entityType': 'person', 'pr...","[Criminal Offense, Domestic Relations, Firearms]",AN ACT RELATING TO CRIMINAL OFFENSES -- WEAPON...,2017-12-31 05:39:37.447119+00:00,"[{'date': '', 'note': '5067', 'links': [{'url'..."
3,[],"[{'date': '2017-02-15', 'description': '02/15/...",\n\n -- S 0288 ======== LC001149 ======== STA...,[bill],2017-07-18 05:33:45.928309+00:00,[],ocd-bill/d844bf20-76b6-4ec1-9e89-0db2312af1a9,SB 288,"{'identifier': '2017', 'jurisdiction': {'name'...",https://openstates.org/ri/bills/2017/SB288,[],"[{'url': 'http://status.rilin.state.ri.us/', '...","[{'name': 'Goodwin', 'entityType': 'person', '...","[Cities & Towns, Police & Fire, Labor]",AN ACT RELATING TO LABOR AND LABOR RELATIONS -...,2017-12-31 05:39:09.603749+00:00,"[{'date': '', 'note': '288', 'links': [{'url':..."
4,[],"[{'date': '2017-03-01', 'description': '03/01/...",\n\n -- H 5802 ======== LC001292 ======== STA...,[bill],2017-07-18 05:33:49.581707+00:00,[],ocd-bill/a9e167c8-5061-453a-97f7-75652dc2fbc5,HB 5802,"{'identifier': '2017', 'jurisdiction': {'name'...",https://openstates.org/ri/bills/2017/HB5802,[],"[{'url': 'http://status.rilin.state.ri.us/', '...","[{'name': 'Handy', 'entityType': 'person', 'pr...","[Health & Safety, Environment, Economic Develo...",AN ACT RELATING TO STATE AFFAIRS AND GOVERNMEN...,2017-12-30 05:39:09.273960+00:00,"[{'date': '', 'note': '5802', 'links': [{'url'..."
5,[],"[{'date': '2017-01-18', 'description': '01/18/...",\n\n -- S 0061 ======== LC000597 ======== STA...,[bill],2017-07-18 05:33:58.496708+00:00,[],ocd-bill/9d921456-5241-4bc0-b604-f7e62b491db3,SB 61,"{'identifier': '2017', 'jurisdiction': {'name'...",https://openstates.org/ri/bills/2017/SB61,[],"[{'url': 'http://status.rilin.state.ri.us/', '...","[{'name': 'Jabour', 'entityType': 'person', 'p...","[Domestic Relations, Marriage]",AN ACT RELATING TO SOLEMNIZATION OF MARRIAGES ...,2017-12-30 05:38:28.040276+00:00,"[{'date': '', 'note': '61', 'links': [{'url': ..."
6,[],"[{'date': '2017-03-01', 'description': '03/01/...",\n\n -- H 5734 ======== LC001652 ======== STA...,[bill],2017-07-18 05:34:03.421577+00:00,[],ocd-bill/5ebc6973-e7a6-41e9-8db2-69a68d13dd81,HB 5734,"{'identifier': '2017', 'jurisdiction': {'name'...",https://openstates.org/ri/bills/2017/HB5734,[],"[{'url': 'http:/

Save our data ...

In [ ]:
pickle.dump(df, open( "rhode_island.pkl", "wb" ))

.
